# Maximum Likelihood Estimate

Suppose we are given a problem where we can assume the _parametric class_ of distribution (e.g. Normal Distribution) that generates a set of data. Since this class of distribution has a finite number of parameters that need to be figured out (mean $\mu$ and standard deviation $\sigma$ in case of normal distribution) in order to identify the particular member of the class, we will use the given data to do so.

The obtained parameter estimates will be called **Maximum Likelihood Estimates**.

Let us consider a Ransom Variable $X$ to be normally distributed with some mena $\mu$ and standard deviation $\sigma$. We need to estimate $\mu$ and $\sigma$ using our samples which accurately represent the actual $X$ and not just the samples that we have drawn out.

## Estimating Parameters

Let's have a look at the Probability Density Function (PDF) for the Normal Distribution and see what they mean.

$$
\begin{equation}
p(x; \mu, \sigma) = \frac{e^{-(x - \mu)^{2}/(2\sigma^{2}) }} {\sigma\sqrt{2\pi}}
\end{equation}
$$ (eq_normal_dist)

This equation is used to obtain the probability of our sample $x$ being from our random variable $X$, when the true parameters of the distribution are $\mu$ and $\sigma$.

Let us consider that our sample = 5. Then what is the probability that it comes from a normal distribution with $\mu = 4$ and $\sigma = 1$? To get this probability, we only need to plug in the values of $x, \mu$ and $\sigma$ in Equation {eq}`eq_normal_dist`. Scipy as a handy function [`norm.pdf()`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.norm.html) that we can use to obtain this as well.

## References

https://medium.com/@rrfd/what-is-maximum-likelihood-estimation-examples-in-python-791153818030